Load results from the `blade_runs` directory and save in a tidy format for R

In [1]:
from os.path import join
import pandas as pd
import numpy as np

In [2]:
from load_and_tidy_posteriors_lib import \
    VALID_METHODS, GetMetadataDataframe, GetMethodDataframe, GetTraceDataframe

In [3]:
base_folder = '/home/rgiordan/Documents/git_repos/DADVI/dadvi-experiments'
input_folder = join(base_folder, 'comparison/blade_runs/')
output_folder = input_folder

folder_method_list = (
    (join(input_folder, "nuts_results/"), 'NUTS'),
    (join(input_folder, "dadvi_results/"), 'DADVI'),
    (join(input_folder, "raabbvi_results/"), 'RAABBVI'),
    (join(input_folder, "sadvi_results/"), 'SADVI'),
    (join(input_folder, "sfullrank_advi_results/"), 'SADVI_FR'),
    #(join(input_folder, "lrvb_results/"), 'LRVB'), # Missing
    #(join(input_folder, 'lrvb_doubling_results'), 'LRVB_Doubling')  # Missing
)


In [4]:
# from load_and_tidy_posteriors_lib import \
#     GetDrawFilenames, LoadPickleSafely, RepList, CheckConvergence, GetEvaluationCount, GetNumDraws, \
#     GetXarrayDatasetScalar

# folder, method = folder_method_list[1]
# print(method, '\n')

# metadata_raw = GetMetadataDataframe(folder, method, return_raw_metadata=True)


DADVI 



In [5]:
# metadata = metadata_raw[1]
# print(metadata.keys())
# # for metadata in metadata_raw:
# #     print(metadata['newton_step_norm'])

dict_keys(['opt_result', 'fixed_draws', 'M', 'kl_hist', 'opt_sequence', 'runtime', 'newton_step_norm', 'newton_step', 'unconstrained_param_names', 'datetime', 'hostname'])


In [6]:
posterior_dfs = []
metadata_dfs = []
trace_dfs = []
for folder, method in folder_method_list:
    print(f'Loading {method}')
    posterior_dfs.append(GetMethodDataframe(folder, method))
    metadata_dfs.append(GetMetadataDataframe(folder, method))
    trace_dfs.append(GetTraceDataframe(folder, method))
    
posterior_df = pd.concat(posterior_dfs)
metadata_df = pd.concat(metadata_dfs)
trace_df = pd.concat(trace_dfs)


Loading NUTS
Loading DADVI
Loading RAABBVI
Loading SADVI
Loading SADVI_FR


In [8]:
posterior_df.to_csv(join(output_folder, 'posteriors_tidy.csv'), index=False)
metadata_df.to_csv(join(output_folder, 'metadata_tidy.csv'), index=False)
trace_df.to_csv(join(output_folder, 'trace_tidy.csv'), index=False)

# Save the names of unconstrained parameters

In [19]:
folder, method = folder_method_list[1]
assert method == 'DADVI'
param_df = GetUnconstraintedParamsDataframe(folder, method)
param_df.to_csv(join(output_folder, 'params_tidy.csv'), index=False)

# Save the full MCMC diagnostic information

In [29]:
folder, method = folder_method_list[0]
assert method == 'NUTS'


draw_filenames, model_names = GetDrawFilenames(folder)
raw_metadata = GetMetadataDataframe(folder, method, return_raw_metadata=True)

from load_and_tidy_posteriors_lib import RepList

mcmc_dict = {
    'model': [],
    'param': [],
    'ess': [],
    'rhat': []
}

for model_ind in range(len(raw_metadata)):
    model = model_names[model_ind]
    metadata = raw_metadata[model_ind]

    for varname in metadata['ess'].data_vars:
        ess = metadata['ess'][varname].values.flatten()
        rhat = metadata['rhat'][varname].values.flatten()
        assert(len(ess) == len(rhat))
        mcmc_dict['model'].append(RepList(model, len(ess)))
        mcmc_dict['param'].append(RepList(varname, len(ess)))
        mcmc_dict['ess'].append(ess)
        mcmc_dict['rhat'].append(rhat)

mcmc_df = pd.DataFrame()
for k,v in mcmc_dict.items():
    mcmc_df[k] = np.hstack(v)

mcmc_df.to_csv(join(output_folder, 'mcmc_diagnostics_tidy.csv'), index=False)

# Explore the contents of the metadata.  

Maybe we want to save additional information.

In [9]:
raw_metadata = {}
for folder, method in folder_method_list:
    print(f'Loading {method}')
    raw_metadata[method] = GetMetadataDataframe(folder, method, return_raw_metadata=True) 


Loading NUTS
Loading DADVI
Loading RAABBVI
Loading SADVI
Loading SADVI_FR


In [10]:
for k,v in raw_metadata.items():
    print('=======================================\n', k, ':')
    print(v[0])
    print('\n')

 NUTS :
{'runtime': 15.77726936340332, 'unconstrained_param_names': ['Intercept', 'treatment', 'pre_test', 'post_test_sigma_log__'], 'ess': <xarray.Dataset>
Dimensions:          (treatment_dim_0: 1, pre_test_dim_0: 1)
Coordinates:
  * treatment_dim_0  (treatment_dim_0) int64 0
  * pre_test_dim_0   (pre_test_dim_0) int64 0
Data variables:
    Intercept        float64 6.553e+03
    treatment        (treatment_dim_0) float64 6.45e+03
    pre_test         (pre_test_dim_0) float64 6.566e+03
    post_test_sigma  float64 5.636e+03, 'rhat': <xarray.Dataset>
Dimensions:          (treatment_dim_0: 1, pre_test_dim_0: 1)
Coordinates:
  * treatment_dim_0  (treatment_dim_0) int64 0
  * pre_test_dim_0   (pre_test_dim_0) int64 0
Data variables:
    Intercept        float64 1.001
    treatment        (treatment_dim_0) float64 1.0
    pre_test         (pre_test_dim_0) float64 1.001
    post_test_sigma  float64 1.001, 'n_chains': 4, 'n_draws': 1000, 'datetime': datetime.datetime(2023, 1, 6, 3, 26, 40, 16